### Classificação multilabel de textos: múltiplos contextos em NLP

Neste projeto vamos criar um modelo de machine learning capaz de classificar as perguntas do Stackoverflow de acordo com seu conteúdo.  

<div style="text-align: center;">
    <img title="a title" alt="Alt text" width="500" height="300" src="perguntas_stackoverflow.jpeg">
</div>

Ao entrarmos no site do [Stackoverflow](https://stackoverflow.com/questions "perguntas") na sessão de perguntas podemos perceber que existem tags/labels nas perguntas. Ao fazermos uma pergunta no site conseguimos colocar essas tags. Desse modo a base de dados do nosso projeto é baseado nas perguntas e tags do Stackoverflow. 

Podemos perceber que estamos lidando com um problema de classificação multilabel pelo fato de termos duas uma mais classes relacionadas a uma pergunta. 

In [1]:
import pandas as pd

perguntas = pd.read_csv("stackoverflow_perguntas.csv")
perguntas.sample(10)

,Perguntas,Tags
3493,"Criei um script chamado 'teste.js"" apenas para...",node.js
4212,Então depois de uma verdadeira peregrinação qu...,jquery
3851,Tenho uma página principal com um formulário s...,jquery html
491,Estou com um problema e acredito que o mesmo e...,angular
4470,Estou com um sistema onde todos os módulos pos...,angular
1608,CODE queria adicionar dentro do array CODE ...,jquery
3573,Como fazer uma paginação sob demanda com Angul...,angular
2844,Estou tendo dificuldades para acessar a api us...,jquery
4558,criei um servidor socket bem simples com NodeJ...,node.js
2404,"Galera, estou procurando esse slider, mas nã...",jquery


In [2]:
perguntas.shape

(5408, 2)

Aqui temos alguns problemas muito comuns quando estamos lidando com strings. Temos algumas strings como o caso da 'html' e 'html ' que pelo ' ' (espaço) são tratadas como diferentes. Ou a 'html angular  ' e 'angular html' em que apenas a ordem está trocada. 

In [3]:
perguntas.Tags.unique()

array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

Nesta célula estamos extraindo os nomes das tags. Podemos perceber que nossas tags se resumem em quatro.

In [4]:
lista_de_tags = list()
for tags in perguntas.Tags.unique():
    for tag in tags.split():
        if tag not in lista_de_tags:
            lista_de_tags.append(tag)
            
lista_de_tags       

['node.js', 'jquery', 'html', 'angular']

É interessante "Binarizarmos" nossas tags já que os algoritmos trabalham com operações matemáticas, ou seja, criar colunas para cada tag e colocar 1 nas linhas onde tiverem perguntas relacionadas com as respectivas tags se não colocar 0. 

In [5]:
def nova_coluna(lista_tags, dataframe, nome_tags):
    for tag in lista_tags:
        coluna = list()
        for linha_tag in dataframe[nome_tags]:
            if tag in linha_tag:
                coluna.append(1)
            else:
                coluna.append(0)
        dataframe[tag] = coluna
nova_coluna(lista_de_tags, perguntas, "Tags")
perguntas.sample(10)

,Perguntas,Tags,node.js,jquery,html,angular
4767,Gostaria de um auxílio no meu problema. Gostar...,jquery,0,1,0,0
4569,"Tenho duas classes A e B, preciso que quando p...",jquery,0,1,0,0
4743,Tenho este arquivo que monta o menu. CODE A...,angular,0,0,0,1
2894,"Tenho uma página de Login, na qual tem um form...",jquery,0,1,0,0
5403,Queria saber como pegar o total de cores de um...,jquery html,0,1,1,0
1296,Galera boa tarde. Como faço uma função em jav...,html,0,0,1,0
4604,Dentro da Factory tem uma variável CODE com v...,angular,0,0,0,1
3111,CODE CODE,jquery html,0,1,1,0
3727,Tenho um link do Click to Call que me retorna ...,jquery,0,1,0,0
5264,Estou definido algumas funções semelhantes par...,jquery,0,1,0,0


Precisamos agrupar nossas tags binarizadas para facilitar a divisão e o treinamento.

In [6]:
lista_zip_tags = list(zip(perguntas[lista_de_tags[0]],
                          perguntas[lista_de_tags[1]],
                          perguntas[lista_de_tags[2]],
                          perguntas[lista_de_tags[3]]))

perguntas["todas_tags"] = lista_zip_tags
perguntas.sample(10)

,Perguntas,Tags,node.js,jquery,html,angular,todas_tags
4356,Tenho um sistema do qual mostra o resultado de...,jquery,0,1,0,0,"(0, 1, 0, 0)"
2281,Gostaria de saber como posso encobrir o endere...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3828,Utilizo a API do highcharts para gerar gráfico...,html,0,0,1,0,"(0, 0, 1, 0)"
5162,Gostaria de saber como em php ou javascript co...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3178,Estou precisando fazer uma validação no campo ...,html,0,0,1,0,"(0, 0, 1, 0)"
3847,"Tenho uma tabela, na última coluna de cada lin...",html,0,0,1,0,"(0, 0, 1, 0)"
78,"Tenho vários registros em PHP, e ao clicar em ...",jquery,0,1,0,0,"(0, 1, 0, 0)"
1275,Não estou conseguindo listas os dados de um Js...,jquery,0,1,0,0,"(0, 1, 0, 0)"
5061,Estou fazendo um aplicativo onde o usuário gra...,node.js,1,0,0,0,"(1, 0, 0, 0)"
4743,Tenho este arquivo que monta o menu. CODE A...,angular,0,0,0,1,"(0, 0, 0, 1)"


Dividindo nossa base de dados.

In [7]:
from sklearn.model_selection import train_test_split

perguntas_treino, perguntas_test, tags_treino, tags_teste = train_test_split(
    perguntas.Perguntas,
    perguntas.todas_tags,
    test_size = 0.2,
    random_state = 123
)

Da mesma forma que termos strings como labels não é interessante, termos as variáveis preditoras como strings também fica difícil de construir alguma coisa. Temos que fazer uma transformação nessas variáveis.

In [8]:
perguntas_treino.head()

1577       array1 = [1,2,3];   array2 = ["um","dois","...
1927    Não sei se fui claro no título, mas quem é da ...
3409    Alguém sabe me dizer qual a melhor forma de re...
4606    Estou com problemas ao tentar validar campos d...
5237    Preciso copiar um valor de dentro de um CODE  ...
Name: Perguntas, dtype: object

Iremos utilizar uma técnica chamada TF-IDF. É a técnica que utilizamos para transformar a linguagem humana em números. É uma técnica que vai dar valores diferentes para as palavras. As que tem significância maior, um grau de distinção entre uma frase e outra, recebe um valor maior. A principal característica do TF-IDF é ser uma pontuação proporcional à frequência da palavra no texto e equilibrada pela frequência no corpus, ou seja, palavras que se repetem muito tendem a ter pontuações menores e são menos relevantes no processo de classificação.

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# max_features é o tamanho do vetor, ou seja, quantas palavras no meu texto 
# max_df = se uma palavra estiver presente em 85% do meu texto, eu desconsidero essa palavra
vetorizar = TfidfVectorizer(max_features=5000, max_df=0.85)
vetorizar

TfidfVectorizer(max_df=0.85, max_features=5000)

In [10]:
vetorizar.fit(perguntas.Perguntas)
perguntas_treino_tfidf = vetorizar.transform(perguntas_treino)
perguntas_test_tfidf = vetorizar.transform(perguntas_test)
print(perguntas_treino_tfidf.shape)
print(perguntas_test_tfidf.shape)

(4326, 5000)
(1082, 5000)


Temos que transformar em um array de listas, o algoritmo não suporta series nem tuplas. 

In [11]:
type(tags_treino)

pandas.core.series.Series

In [12]:
import numpy as np

tags_treino_array = np.asarray(list(tags_treino))
tags_teste_array = np.asarray(list(tags_teste))
print(tags_treino_array)
print(type(tags_treino_array))

[[0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 ...
 [0 1 1 0]
 [0 0 1 0]
 [0 1 1 0]]
<class 'numpy.ndarray'>


Essa acuracia é medida pelo acerto da linha completa, ou seja, de todas as tags. Também chamamos de Exact match.

In [13]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

regressao_logistica = LogisticRegression()
classificador_onevsrest = OneVsRestClassifier(regressao_logistica)
classificador_onevsrest.fit(perguntas_treino_tfidf, tags_treino_array)

resultado_onevsrest = classificador_onevsrest.score(perguntas_test_tfidf, tags_teste_array)
print("Resultado {0: .2f}%".format(resultado_onevsrest*100))

Resultado  41.68%


Outra técnica de avaliação do nosso modelo é o Hamming Loss. Este avalia os acertos de cada label individualmente. Diferente da acurácia, aqui quanto mais próximo de zero melhor. 

Utilizaremos a técnica de avaliação que mais fizer sentido no contexto do nosso problema.

In [14]:
from sklearn.metrics import hamming_loss

# o score() só da o resultado da predição, aqui precisamos saber as presições: 
previsao_onevsrest = classificador_onevsrest.predict(perguntas_test_tfidf) 
hamming_loss_onevsrest = hamming_loss(tags_teste_array, previsao_onevsrest)
print("Hamming Loss {0: .2f}".format(hamming_loss_onevsrest))   

Hamming Loss  0.19


In [15]:
#!pip install scikit-learn==0.24.1 --user

Outra opção para realizarmos a classificação é utizar a Classificação em cadeia. Da mesma maneira que o modelo de relevãncia binária a classificação em cadeia vai criar um modelo para cada coluna. O que vai mudar é a construção da base de treinamento. Uma vez que a primeira coluna é treinada ela também vai servir de imput para classificar a segunda coluna o resultado do treinamento também vai servir de imput para classificar a terceira coluna e assim sucessivamente. 

In [16]:
from skmultilearn.problem_transform import ClassifierChain

classificador_cadeia = ClassifierChain(regressao_logistica)
classificador_cadeia.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_cadeia = classificador_cadeia.score(perguntas_test_tfidf, tags_teste_array)
previsao_cadeia = classificador_cadeia.predict(perguntas_test_tfidf)
hamming_loss_cadeia = hamming_loss(tags_teste_array, previsao_cadeia)
print("Hamming Loss {0: .2f}".format(hamming_loss_cadeia))
print("Resultado {0: .2f}%".format(resultado_cadeia*100))

Hamming Loss  0.21
Resultado  49.82%


Temos uma outra opção que também se adequa ao nosso problema. O mlknn é um dos algoritmo adaptado para solucionar o problema de classificação multilabel. Um knn adaptado para classificação multilabel.

In [17]:
from skmultilearn.adapt import MLkNN

classificador_mlknn = MLkNN()
classificador_mlknn.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_mlknn = classificador_mlknn.score(perguntas_test_tfidf, tags_teste_array)
previsao_mlknn = classificador_mlknn.predict(perguntas_test_tfidf)
hamming_loss_mlknn = hamming_loss(tags_teste_array, previsao_mlknn)
print("Hamming Loss {0: .2f}".format(hamming_loss_mlknn))
print("Resultado {0: .2f}%".format(resultado_mlknn*100))

C:\Users\BlueShift\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Hamming Loss  0.25
Resultado  32.53%


Organizando os nossos resultados para termos uma melhor visualização dos nossos resultados. 

In [19]:
resultados_classificacao = pd.DataFrame()
resultados_classificacao["perguntas"] = perguntas_test.values
resultados_classificacao["tags real"] = list(tags_teste) # transformando em lista para "zerar o index" que foi randomizado pelo train_test_split
resultados_classificacao["cadeia"] = list(previsao_cadeia.toarray())
resultados_classificacao["mlknn"] = list(previsao_mlknn.toarray())
resultados_classificacao
     

,perguntas,tags real,cadeia,mlknn
0,estou com conflito entre o CODE e os CODE ...,"(0, 1, 0, 0)","[0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 0]"
1,Estou fazendo um site que eu sou obrigado a us...,"(0, 0, 1, 0)","[0.0, 0.0, 1.0, 0.0]","[0, 1, 1, 0]"
2,Recentemente fiz um refactor do meu código par...,"(1, 0, 0, 0)","[1.0, 0.0, 0.0, 0.0]","[1, 0, 0, 0]"
3,Eu tenho esse código em CODE que passo valore...,"(0, 1, 1, 0)","[0.0, 1.0, 0.0, 0.0]","[0, 1, 1, 0]"
4,"Olá, em minha função tem o evento CODE que de...","(0, 1, 1, 0)","[0.0, 1.0, 0.0, 0.0]","[0, 1, 1, 0]"
...,...,...,...,...
1077,Estou a desenvolver um website em jQuery. E at...,"(0, 1, 0, 0)","[0.0, 1.0, 1.0, 0.0]","[0, 0, 1, 0]"
1078,Estou usando este plugin - jquery autocomplete...,"(0, 1, 0, 0)","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]"
1079,"Tenho o seguinte jQuery: CODE Nisto, quanti...","(0, 1, 0, 0)","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]"
1080,Estou usando o SimpleModal Contact Form de Eri...,"(0, 1, 0, 0)","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]"


### Teste isolado

In [24]:
perguntas.head(2)

,Perguntas,Tags,node.js,jquery,html,angular,todas_tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0,"(1, 0, 0, 0)"
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0,"(1, 0, 0, 0)"


In [25]:
resultados_classificacao.perguntas[1]

'Estou fazendo um site que eu sou obrigado a usar HTML, CSS e JavaScript no máximo. Só que muitas coisas que eu preciso do PhP eu não posso usar. Sendo assim eu estive pesquisando por sinonimo de CODE  e encontrei o seguinte, que ajudou em partes:  CODE   Tá, mas qual é o problema? Ele cria uma barra de rolagem do lado. Eu sei como remover com CSS, mas o texto que foi inserido rola só dentro da área da CODE  e eu queria que fizesse como se o texto inserido fizesse parte da página, como funciona no CODE  do PhP. A questão é, como fazer a mecânica do JavaScript nesse caso ficar igual ao do PhP?  Print para exemplificar: https://imgur.com/a/V9AGLgB '

In [26]:
resultados_classificacao.iloc[1]

perguntas    Estou fazendo um site que eu sou obrigado a us...
tags real                                         (0, 0, 1, 0)
cadeia                                    [0.0, 0.0, 1.0, 0.0]
mlknn                                             [0, 1, 1, 0]
Name: 1, dtype: object

Neste exemplo percebemos que o método de Classificação em Cadeia classificou/"tagueou" corretamente como HTML a pergunta feita. Já o MLKNN além de classificar como HTML também classificou como jquery, erroneamente comparado com verdadeiro label.